# Retention Daily Call data

In [20]:
# importing four(BIGTABLE(Details),DARK REPORT(Dark),SEGMENTATION(df),OPEN CASES (Opcase)) tables needed for Retention daily Drive(3,18,34 days in dark) by reading all of the workbook into pandas)
# importing table 'SEGMENTATION(df)' and read as pandas.....
import pandas as pd

Segement = pd.read_csv('segement.csv')

In [22]:
#importing table 'BIGTABLE(Details)'and read as pandas.....
Bigtable = pd.read_csv('Bigtable.csv',usecols = ['Full SFID','Contract Id','Type','Customer Signed By: Full Name','Customer Signed By: Phone','Customer: Secondary Phone number','Customer Warranty End Date','Price Book: Price Book Name','Contract Start Date','Status','StatusII','Customer Status'])

C:\Users\Joshua.Aiyeetan\AppData\Local\Temp\ipykernel_15784\3274425705.py:2: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  Bigtable = pd.read_csv('Bigtable.csv',usecols = ['Full SFID','Contract Id','Type','Customer Signed By: Full Name','Customer Signed By: Phone','Customer: Secondary Phone number','Customer Warranty End Date','Price Book: Price Book Name','Contract Start Date','Status','StatusII','Customer Status'])


In [3]:
#importing table 'OPEN CASES (Opcase)'and read as pandas....
Opcase = pd.read_csv('Legal_FS_Open_Case.csv')

In [25]:
#importing table 'DARK REPORT'(Dark) and read as pandas.....
#Reading the required excel sheets and columns from the BI Daily Reports. The selected columns are considered valid for the required task
Contract = pd.read_excel('BI NG daily reports of 2023-04-01.xlsx', 'Contracts', usecols = ['Contract','SaleCH','generation','LTOperiod','LTOunits','type'])

 Sps_Contract = pd.read_excel('BI NG daily reports of 2023-04-01.xlsx','Sps Contracts', usecols = ['Contract','activation_start','activation_end'])

Paidperiod = pd.read_excel('BI NG daily reports of 2023-04-01.xlsx','Contracts per PaidPeriod', usecols = ['Contract','PaidPeriods'])

r4 = pd.read_excel('BI NG daily reports of 2023-04-01.xlsx','CRM contracts', names = ['Contract','CRMcontract'])

r5 = pd.read_excel('BI NG daily reports of 2023-04-01.xlsx','Last payments', usecols = ['Contract','payment_date','activation_end'])

r5.columns = ['Contract','last_payment_date','lastpayment_activation_end']

In [26]:
check

Timestamp('2023-01-03 00:00:00')

In [5]:
r2['activation_start'] = pd.to_datetime(r2['activation_start']).dt.normalize()
r2['activation_end'] = pd.to_datetime(r2['activation_end']).dt.normalize()
r5['lastpayment_activation_end'] = pd.to_datetime(r5['lastpayment_activation_end'],errors = 'coerce')
r5['last_payment_date'] = pd.to_datetime(r5['last_payment_date']).dt.normalize()
r5['lastpayment_activation_end'] = pd.to_datetime(r5['lastpayment_activation_end']).dt.normalize()

In [6]:
check1 = pd.merge(pd.merge(pd.merge(pd.merge(r1, r2, how='left', on='Contract'), r3, how='left', on='Contract'), r4, how='left', on='Contract'), r5, how='left', on='Contract')
check1 = pd.merge(check1, Details, how='left', left_on='CRMcontract', right_on='Contract Id')

check1 = pd.merge(pd.merge(check1, df[['Contract Id','Potential Grouping Other']], how='left', left_on='CRMcontract', right_on='Contract Id'), Opcase[['Contract: Full SFID','Department','Subject']], how='left', left_on='Contract', right_on='Contract: Full SFID')

check1 = check1[check1['Type'] == 'Production']

check1["ref_activation_end"] = check1[["activation_end","lastpayment_activation_end"]].max(axis=1)

check1['ref_activation_end'] = check1['ref_activation_end'].fillna('1900-01-01')

check1['ref_activation_end'] = pd.to_datetime(check1['ref_activation_end']).dt.normalize()

In [7]:
import pandas as pd
import datetime as date
from datetime import date
from datetime import timedelta

today = pd.to_datetime('today').normalize() - timedelta(days=1)

check1['Outage_days'] = (check1['ref_activation_end'] - today).dt.days

In [8]:
#Contract Start Date
# A date format was applied to both ContractCreation and Customer Warranty End Date. 
# Also normalized by removing the time stamp
check1['Customer Warranty End Date'] = pd.to_datetime(check1['Customer Warranty End Date'], format='%d/%m/%Y')

check1['Contract Start Date'] = pd.to_datetime(check1['Contract Start Date'], format='%d/%m/%Y')

In [9]:
#Applying  elif conditional statement to achieved customer within and outside warranty. 
#inaddition, to avoid timestamp and stir error date was converted datetime in pandas 
import datetime as date

check = pd.to_datetime('01/03/2023')

def warranty(i):
    if i >= check:
        return 'Within Warranty'
    elif i == ' ':
        return 'N/A'
    else:
        return 'Outside Warranty'
    

check1['Warranty Status'] = check1['Customer Warranty End Date'].apply(warranty)

In [10]:
import numpy as np
filters = [
   (check1['Price Book: Price Book Name'].str.contains("NWO") == True)
]
values = ['LCP']

check1["lcp_status"] = np.select(filters, values, default='Non-LCP')

In [11]:
# Using filters as conditional statement on Join2 coloumn='Days_in_Dark' to achieved the exact numbers of required dark days in sentence for report.
# However, an other coloumns='Dark_Days' was created on the table to identify the required dark days
import numpy as np

filters = [
    (check1.Outage_days == -1),
   (check1.Outage_days == -3),
    (check1.Outage_days == -4),
    (check1.Outage_days == -7),
    (check1.Outage_days == -10),
    (check1.Outage_days == -13),
    (check1.Outage_days == -17),
   (check1.Outage_days == -18),
   (check1.Outage_days == -30),
    (check1.Outage_days == -35)]
values = ['1 DarkDay','3 DarkDays','4 DarkDays','7 DarkDays','10 DarkDays','13 DarkDays','17 DarkDays', '18 DarkDays', '30 DarkDays', '35 DarkDays']

check1["Dark_Days"] = np.select(filters, values, default= check1['Outage_days'])

In [12]:
# Using filters as conditional statement on Join2 coloumn='Department' to achieved the required Open cases Pool Status(CSS Pool, FS Pool, Legal Pool) in sentence for report.
# However, an other coloumns='Pool Status' was created on the table to identify the required Customer open cases Pool
import numpy as np

filters = [
   (check1.Department == 'NG Legal'),
   (check1.Department == 'NG FS') & (check1.Subject == 'Requests / Cancel membership')
]
values = ['Legal Pool', 'FS Pool']


check1["Pool Status"] = np.select(filters, values, default= 'CSS Pool')

In [13]:
import numpy as np

filters = [
   (check1.Outage_days == -1) & (check1['Potential Grouping Other'].str.contains("Good") == True),
    (check1.Outage_days == -7) & (check1['Potential Grouping Other'].str.contains("Good") == True),
    (check1.Outage_days == -13) & (check1['Potential Grouping Other'].str.contains("Good") == True),
    (check1.Outage_days == -18) & (check1['Potential Grouping Other'].str.contains("Good") == True),
    (check1.Outage_days == -30) & (check1['Potential Grouping Other'].str.contains("Good") == True),
    (check1.Outage_days == -3) & (check1['Potential Grouping Other'].str.contains("Fair") == True),
    (check1.Outage_days == -10) & (check1['Potential Grouping Other'].str.contains("Fair") == True),
    (check1.Outage_days == -17) & (check1['Potential Grouping Other'].str.contains("Fair") == True),
    (check1.Outage_days == -35) & (check1['Potential Grouping Other'].str.contains("Fair") == True),
    (check1.Outage_days == -4) & (check1['Potential Grouping Other'].str.contains("Challenging") == True),
    (check1.Outage_days == -13) & (check1['Potential Grouping Other'].str.contains("Challenging") == True),
    (check1.Outage_days == -18) & (check1['Potential Grouping Other'].str.contains("Challenging") == True),
    (check1.Outage_days == -35) & (check1['Potential Grouping Other'].str.contains("Challenging") == True),
    (check1.Outage_days == -35) & (check1['Potential Grouping Other'].str.contains("Bad") == True),
    (check1.Outage_days == -35) & (check1['Potential Grouping Other'].str.contains("Good") == True)
    ]
values = ['1Good','7Good','13Good','18Good','30Good','3Fair','10Fair','17Fair','35Fair','4Challenging','13Challenging','18Challenging','35Challenging','35Bad','35Good']


check1["Retention Status"] = np.select(filters, values, default= 'Not Needed')

In [14]:
#Rearrangement of entire coloumns header to suit the report requirement [Contract Short Num	Consecutive No Activity Days	eula_free_to_use_days	Customer name	Payer number	Secondary Number	Contract Model Type	Contract Start Date	Generation	Dark Days	Warranty End Date	Warranty Status @as 1st Nov'21	Pool Status	Customer Segmentation]
# However, a new variable Join3 was created via Join2 variable. 

check2 = check1[['CRMcontract','Outage_days','LTOperiod','Type','Customer Signed By: Full Name','Customer Signed By: Phone',
               'Customer: Secondary Phone number','type','Contract Start Date','SaleCH','generation','Dark_Days','Customer Warranty End Date',
               'Warranty Status','Pool Status','Potential Grouping Other','Price Book: Price Book Name','Status','StatusII','Customer Status','Retention Status']]

In [15]:
# Selection of the exact dark days('-1,-3',-4',-7',-10',-13',-17',-18','-30','-35') and contract type('production') from Join3 variable table
check3 = check2[((check2['Outage_days'] == -1) | (check2['Outage_days'] == -3)| (check2['Outage_days'] == -4)| (check2['Outage_days'] == -7)| (check2['Outage_days'] == -10)| (check2['Outage_days'] == -13)| (check2['Outage_days'] == -17) | (check2['Outage_days'] == -18) | (check2['Outage_days'] == -30) | (check2['Outage_days'] == -35)) & ((check2['Price Book: Price Book Name'] != 'Direct-NG (NWO)') & (check2['Status'] != 'Cancelled') & (check2['StatusII'] != 'Freeing') & (check2['Customer Status'] != 'Owner') & (check2['Status'] != 'Canceled') & (check2['StatusII'] != 'Free') & (check2['Type'] == 'Production')& (check2['Retention Status'] != 'Not Needed'))]

    
check3.columns = ['Contract Short Num','Consecutive No Activity Days','Type','Eula_Free_to_Use_Days','Customer Name',
               'Payer Number','Secondary Number','Contract Model Type','Contract Start Date','Tenant','Generation','Dark Days',
               'Warranty End Date','Warranty Status','Pool Status','Customer Segmentation','Price Book: Price Book Name','Status','StatusII','Customer Status','Retention Status']

In [16]:
# dropping the coloumn="type" which is not needed in the table.
check3.drop('Type', axis = 1, inplace=True)
check3.drop('Price Book: Price Book Name', axis = 1, inplace=True)
check3.drop('Status', axis = 1, inplace=True)
check3.drop('StatusII', axis = 1, inplace=True)
check3.drop('Customer Status', axis = 1, inplace=True)
check3.drop('Retention Status', axis = 1, inplace=True)

C:\Users\Joshua.Aiyeetan\AppData\Local\Temp\ipykernel_15784\2948411428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check3.drop('Type', axis = 1, inplace=True)
C:\Users\Joshua.Aiyeetan\AppData\Local\Temp\ipykernel_15784\2948411428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check3.drop('Price Book: Price Book Name', axis = 1, inplace=True)
C:\Users\Joshua.Aiyeetan\AppData\Local\Temp\ipykernel_15784\2948411428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [17]:
# writing the final output into an excel file
check3.to_excel('Retention Call Drive New1.xlsx')

In [18]:
# grouped by Generation and Dark days to get summary counts
check3.groupby(['Customer Segmentation','Dark Days']).count()['Contract Short Num']

Customer Segmentation  Dark Days  
Bad                    35 DarkDays     22
Challenging            13 DarkDays    134
                       18 DarkDays     91
                       35 DarkDays     63
                       4 DarkDays     264
Fair                   10 DarkDays     66
                       17 DarkDays     51
                       3 DarkDays     172
                       35 DarkDays     22
Good                   1 DarkDay        4
Name: Contract Short Num, dtype: int64